# Creating sample

In [ ]:
from pathlib import Path
datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

from pylmgc90 import pre

In [ ]:
# container definition:
#   * of bodies
bodies = pre.avatars()
#   * of materials
mats = pre.materials()
#   * of models
mods = pre.models()
#   * of visibility tables
svs = pre.see_tables()
#   * of contact laws
tacts = pre.tact_behavs()

# on se place en 3D
dim = 3

In [ ]:
# rigid material
tdur = pre.material(name='TDURx', materialType='RIGID', density=2500.)
mats.addMaterial(tdur)

# rigid model
mod = pre.model(name='rigid', physics='MECAx', element='Rxx3D', dimension=dim)
mods.addModel(mod)

In [ ]:
!gmsh quart_voute.msh

In [ ]:
# reading msh file
complete_mesh = pre.readMesh(name='quart_voute.msh', dim=dim)

xyz=[18,25,32,39,57,64,71,78]
byz=[1,2,3,4,5,6,7,8,9,10,11]
bxz=[40,41,42,43,44,45,46,47,48,49,50]
yz=[51,58,65,72]
xz=[12,19,26,33]

# separating meshes
meshes = complete_mesh.separateMeshes(dim=dim,entity_type="geometricalEntity") #, keep_all_elements=True)

# building avatars
for i in meshes.keys():
    # boundary condition
    if int(i) in xyz:
        body = pre.volumicMeshToRigid3D(volumic_mesh=meshes[i], model=mod, material=tdur, color='REDxx')   
        body.imposeDrivenDof(component=[1, 2, 3, 4, 5, 6], dofty='vlocy')   
    elif int(i) in yz:   
        body = pre.volumicMeshToRigid3D(volumic_mesh=meshes[i], model=mod, material=tdur, color='REDxx')           
        body.imposeDrivenDof(component=[2, 3, 4, 5, 6], dofty='vlocy')
    elif int(i) in xz:
        body = pre.volumicMeshToRigid3D(volumic_mesh=meshes[i], model=mod, material=tdur, color='REDxx')   
        body.imposeDrivenDof(component=[1, 3, 4, 5, 6], dofty='vlocy')      
    elif int(i) in byz or int(i) in bxz:
        continue  
    else:
        body = pre.volumicMeshToRigid3D(volumic_mesh=meshes[i], model=mod, material=tdur, color='BLUEx')   

    bodies.addAvatar(body)


In [ ]:
# definitions des interactions
#  * loi d'interaction :
lprpr = pre.tact_behav('iqsG0', 'IQS_CLB_g0', fric=0.3)
tacts+= lprpr

#  * table de visibilite :
sv1 = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='POLYR', colorCandidat   ='BLUEx',
                    CorpsAntagoniste='RBDY3', antagoniste='POLYR', colorAntagoniste='BLUEx',
                    behav='iqsG0', alert=2.5e-2)
svs+=sv1
sv2 = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='POLYR', colorCandidat   ='BLUEx',
                    CorpsAntagoniste='RBDY3', antagoniste='POLYR', colorAntagoniste='REDxx',
                    behav='iqsG0', alert=2.5e-2)
svs+=sv2

In [ ]:
post = pre.postpro_commands()
post.addCommand( pre.postpro_command(name='SOLVER INFORMATIONS', step=1))
post.addCommand( pre.postpro_command(name='VIOLATION EVOLUTION', step=1))

In [ ]:
# ecriture des fichiers de donnees
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox)

In [ ]:
pre.visuAvatars(bodies)

# Computing

In [ ]:
from pylmgc90.chipy import computation

In [ ]:
dim = 3

# time evolution parameters
dt = 1e-3
nb_steps = 500

# theta integrator parameter
theta = 0.5

# nlgs parameters
tol    = 1e-4
relax  = 1.0
norm   = 'Quad '
gs_it1 = 500
gs_it2 = 10
stype  = 'Stored_Delassus_Loops         '

# write parameter¶
freq_write   = 10

# display parameters
freq_display = 10

In [ ]:
from pylmgc90 import chipy

chipy.POLYR_SkipAutomaticReorientation()
chipy.POLYR_FlatnessAngle(10.)
chipy.POLYR_TopologyAngle(3.)

chipy.PRPRx_ShrinkPolyrFaces(1e-3)
#chipy.PRPRx_UseCpF2fExplicitDetection(1e-1)
chipy.PRPRx_UseNcF2fExplicitDetection(5e-1,1e-1)
chipy.PRPRx_LowSizeArrayPolyr(100)

chipy.nlgs_3D_DiagonalResolution()

computation.initialize(dim, dt, theta, logmes=True)

for k in range( 1, nb_steps+1 ):
    if k%50 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, tol ,relax, gs_it1, gs_it2, freq_write, freq_display)
computation.finalize()

In [ ]:
#!paraview